In [18]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

llm = init_chat_model("openai:gpt-4o")

In [19]:
class Ingredient(BaseModel):

    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):

    ingredients: list[Ingredient]


class State(TypedDict):

    dish: str
    ingredients: list[Ingredient]
    recipe_steps: str
    plating_instructions: str

In [20]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    response = structured_llm.invoke(
        f"List 5-8 ingredients needed to make {state['dish']}"
    )
    return {"ingredients": response.ingredients}


def create_recipe(state: State):
    response = llm.invoke(
        f"Write a step by step cooking intruction for {state['dish']}, using these ingredients {state['ingredients']}"
    )

    return {"recipe_steps": response.content}


def describe_plating(state: State):
    response = llm.invoke(
        f"Describe how to beautifully plate this dish {state['dish']} based on this recipe {state['recipe_steps']}"
    )

    return {"plating_instructions": response.content}

In [21]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)

graph = graph_builder.compile()

In [22]:
graph.invoke({"dish": "hummus"})

{'dish': 'hummus',
 'ingredients': [Ingredient(name='Chickpeas', quantity='2', unit='cups'),
  Ingredient(name='Tahini', quantity='1/4', unit='cup'),
  Ingredient(name='Lemon juice', quantity='2', unit='tablespoons'),
  Ingredient(name='Garlic cloves', quantity='2', unit='pieces'),
  Ingredient(name='Olive oil', quantity='2', unit='tablespoons'),
  Ingredient(name='Cumin', quantity='1', unit='teaspoon'),
  Ingredient(name='Salt', quantity='1', unit='teaspoon'),
  Ingredient(name='Water', quantity='2-4', unit='tablespoons')],
 'recipe_steps': "Here's a step-by-step guide to making delicious hummus using your list of ingredients:\n\n### Ingredients:\n- 2 cups Chickpeas (cooked or canned)\n- 1/4 cup Tahini\n- 2 tablespoons Lemon juice\n- 2 Garlic cloves\n- 2 tablespoons Olive oil\n- 1 teaspoon Cumin\n- 1 teaspoon Salt\n- 2-4 tablespoons Water (as needed)\n\n### Instructions:\n\n1. **Prepare Chickpeas**:\n   - If using canned chickpeas, drain and rinse them under cold water.\n   - If using